In [3]:
%load_ext autoreload
%autoreload 2

import sympy 
import numpy as np 
import pandas as pd 

from utilities.circuit_database import CirqTranslater
from utilities.templates import *
from utilities.variational import Minimizer
from utilities.misc import get_qubits_involved, reindex_symbol
import matplotlib.pyplot as plt 


In [4]:
def get_positional_dbs(circuit, circuit_db):

    qubits_involved = get_qubits_involved(circuit, circuit_db)
    
    gates_on_qubit = {q:[] for q in qubits_involved}
    on_qubit_order = {q:[] for q in qubits_involved}

    for order_gate, ind_gate in enumerate( circuit_db["ind"]):
        if ind_gate < translator.number_of_cnots:
            control, target = translator.indexed_cnots[str(ind_gate)]
            gates_on_qubit[control].append(ind_gate)
            gates_on_qubit[target].append(ind_gate)
            on_qubit_order[control].append(order_gate)                
            on_qubit_order[target].append(order_gate)  
        else:
            gates_on_qubit[(ind_gate-translator.n_qubits)%translator.n_qubits].append(ind_gate)
            on_qubit_order[(ind_gate-translator.n_qubits)%translator.n_qubits].append(order_gate)        
    return gates_on_qubit, on_qubit_order

In [7]:
def rule_1(translator, simplified_db, on_qubit_order, gates_on_qubit):
    simplification = False
    
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path):
            if ind_gate < translator.number_of_cnots:
                control, target = translator.indexed_cnots[str(ind_gate)]
                if (q == control) and (order_gate_on_qubit == 0):
                    pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                    
                    block_id = circuit_db.iloc[pos_gate_to_drop]["block_id"]
                    simplified_db.loc[int(pos_gate_to_drop)+0.1] = gate_template(translator.number_of_cnots + translator.n_qubits + control, param_value=0.0, block_id=circuit_db.iloc[0]["block_id"])
                    simplified_db.loc[int(pos_gate_to_drop)+0.11] = gate_template(translator.number_of_cnots + translator.n_qubits + target, param_value=0.0, block_id=circuit_db.iloc[0]["block_id"])
                    
                    simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                    
                    simplification = True
                    break
    simplified_db = simplified_db.sort_index().reset_index(drop=True)
    return simplification, simplified_db


def rule_2(translator, simplified_db, on_qubit_order, gates_on_qubit):
    simplification = False
    
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            
            next_ind_gate = qubit_gates_path[order_gate_on_qubit+1]
            if (ind_gate < translator.number_of_cnots) and (ind_gate == next_ind_gate):
                control, target = translator.indexed_cnots[str(ind_gate)]
                not_gates_in_between = False
                this_qubit = q
                other_qubits = [control, target]
                other_qubits.remove(q)
                other_qubit = other_qubits[0]
                
                ## now we need to check what happens in the other_qubit
                for qord_other, ind_gate_other in enumerate(gates_on_qubit[other_qubit][:-1]):
                    if (ind_gate_other == ind_gate) and (gates_on_qubit[other_qubit][qord_other +1] == ind_gate):
                        ## if we append the CNOT for q and other_q on the same call, and also for the consecutive
                        ## note that in between there can be other calls for other qubits
                        order_call_q = on_qubit_order[q][order_gate_on_qubit]
                        order_call_other_q = on_qubit_order[other_qubit][qord_other]
                        
                        order_call_qP1 = on_qubit_order[q][order_gate_on_qubit+1]
                        order_call_other_qP1 = on_qubit_order[other_qubit][qord_other+1]
                        
                        ## then it's kosher to say they are consecutively applied (if only looking at the two qubits)
                        if (order_call_q == order_call_other_q) and (order_call_qP1 == order_call_other_qP1):
                            
                            pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                            simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                            pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit+1]
                            simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)

                            simplification = True
                            break
                if simplification is True:
                    break
    simplified_db = simplified_db.reset_index(drop=True)
    return simplification, simplified_db



def apply_rule(original_circuit_db, rule, **kwargs):
    max_cnt = kwargs.get('max_cnt',10)
    simplified, cnt = True, 0
    original_circuit, original_circuit_db = translator.give_circuit(original_circuit_db)
    gates_on_qubit, on_qubit_order = get_positional_dbs(original_circuit, original_circuit_db)
    simplified_db = original_circuit_db.copy()
    while simplified and cnt < max_cnt:
        simplified, simplified_circuit_db = rule(translator, simplified_db, on_qubit_order, gates_on_qubit)
        circuit, simplified_db = translator.give_circuit(simplified_circuit_db)
        gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, simplified_db)
        cnt+=1
    return cnt, simplified_db

In [97]:
def rule_3(translator, simplified_db, on_qubit_order, gates_on_qubit):
    simplification = False
    for q, qubit_gates_path in gates_on_qubit.items():
        if simplification is True:
            break
        for order_gate_on_qubit, ind_gate in enumerate(qubit_gates_path[:-1]):
            if order_gate_on_qubit == 0 and (translator.number_of_cnots <= ind_gate< translator.number_of_cnots+ translator.n_qubits ):
                pos_gate_to_drop = on_qubit_order[q][order_gate_on_qubit]
                simplified_db = simplified_db.drop(labels=[pos_gate_to_drop],axis=0)
                simplified_db = simplified_db.reset_index(drop=True)
                simplified_db = shift_symbols_down(translator, pos_gate_to_drop, simplified_db)
                simplification = True
                break
    return simplification, simplified_db

In [103]:
translator = CirqTranslater(3)
db1 = u1_layer(translator)
db2 = cnot_layer(translator, block_id = 2)
circuit_db = concatenate_dbs([db1, db2, db1])
gates_on_qubit, on_qubit_order = get_positional_dbs(circuit, circuit_db)

In [104]:
circuit, circuit_db  = translator.give_circuit(circuit_db)
cnts, simplified_db = apply_rule(simplified_db, rule_3)

In [94]:
ss = circuit_db.copy()

In [95]:
cnts, simplified_db = rule_3(translator, circuit_db, on_qubit_order, gates_on_qubit)

In [203]:
circuit, circuit_db = translator.give_circuit(simplified_db)

1

3

In [201]:
circuit, circuit_db = translator.give_circuit(simplified_db)

12

In [96]:
simplified_db

,ind,symbol,param_value,trainable,block_id
0,9,th_0,None,True,0
1,6,th_1,None,True,0
2,7,th_2,None,True,0
3,10,th_3,None,True,0
4,7,th_4,None,True,0
5,8,th_5,None,True,0
6,11,th_6,None,True,0
7,8,th_7,None,True,0
8,0,None,None,True,2
9,3,None,None,True,2


In [79]:
simplified_db

,ind,symbol,param_value,trainable,block_id
0,9,th_1,None,True,0
1,6,th_1,None,True,0
2,7,th_2,None,True,0
3,10,th_3,None,True,0
4,7,th_4,None,True,0
5,8,th_5,None,True,0
6,11,th_6,None,True,0
7,8,th_7,None,True,0
8,0,None,None,True,0
9,3,None,None,True,0


KeyError: '[0] not found in axis'

In [58]:
ss

,ind,symbol,param_value,trainable,block_id
1,9,th_1,None,True,0
2,6,th_2,None,True,0
3,7,th_3,None,True,0
4,10,th_4,None,True,0
5,7,th_5,None,True,0
6,8,th_6,None,True,0
7,11,th_7,None,True,0
8,8,th_8,None,True,0
9,0,None,None,True,0
10,3,None,None,True,0


,ind,symbol,param_value,trainable,block_id
0,6,th_0,None,True,0
1,9,th_1,None,True,0
2,6,th_2,None,True,0
3,7,th_3,None,True,0
4,10,th_4,None,True,0
5,7,th_5,None,True,0
6,8,th_6,None,True,0
7,11,th_7,None,True,0
8,8,th_8,None,True,0
9,0,None,None,True,0
